In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [36]:
URL = 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page='

data = requests.get(URL).text

In [5]:
type(data)

str

In [37]:
with open('./data/mercadolivreoferdia.html','w', encoding='utf-8') as f:
    f.write(data)

In [43]:
def get_items_title(doc):
    """
        Return the title of products from tags
    """
    title_tags = doc.find_all('p',class_='promotion-item__title')
    titles = []

    for tags in title_tags:
        titles.append(tags.text)

    return titles

In [50]:
# BUG - Considering the previous price as a real price, duplicating the array of data.
def get_items_price(doc):
    """
        Return the price of products from tags
    """
    price_tags = doc.find_all('span',class_='andes-money-amount__fraction')
    prices = []

    for tags in price_tags:
        prices.append(tags.text)

    return prices

In [71]:
def get_doc(url):
    """
        Return the html parsed instance.
    """
    time.sleep(1)
    response =  requests.get(url)
    doc = BeautifulSoup(response.text,'html.parser')

    if response.status_code != 200:
        raise Exception("Failure to load the page!") 

    return doc

In [61]:
def scrape_multiple_pages(n):
    titles, prices = [] , [] 

    for page in range(1, n+1):
        doc = get_doc(URL + str(page))
        titles.extend(get_items_title(doc))
        prices.extend(get_items_price(doc))

    items = {
        'TITLE': titles,
        'PRICE': prices
    }

    return pd.DataFrame.from_dict(items,orient='index').transpose()

In [72]:
df = scrape_multiple_pages(10)

df.head()

,TITLE,PRICE
0,Motorola Moto G04 Grafite 128gb Dual Sim 4gb R...,699
1,Samsung Smart Tv 43 Crystal Uhd 4k 43du8000 2024,849
2,Geladeira frost free Electrolux DFN41 branca c...,1.815
3,Notebook Dell Inspiron I15-i120k-a30pf I5 16gb...,2.199
4,Loção hidratante Cerave Com Ácido Hialurônico ...,2.699


In [73]:
df.to_csv("./data/mercado_livre.csv")